In [5]:
import os
import plotly.express as px

import matplotlib.pyplot as plt

import basd
from matplotlib import pyplot
import numpy as np
import pandas as pd
import rioxarray as rio
import seaborn as sns
import xarray as xr

#### Reading NetCDF file
Reading in observational precipitation data from 1979-2014 supplied in the data directory.

In [6]:
pr_obs_hist = xr.open_dataset('../basd/data/pr_obs-hist_fine_1979-2014.nc')
pr_sim_hist = xr.open_dataset('../basd/data/pr_sim-hist_coarse_1979-2014.nc')
pr_sim_fut = xr.open_dataset('../basd/data/pr_sim-fut_coarse_2065-2100.nc')

### Running Bias Adjustment
#### Initializing our first `Adjustment` object
Here we are providing the three required data arrays and a variable name 'pr' representing the daily mean precipitation variable. Precipitation is assumed to follow a gamma distribution, and thus requires a lower bound and lower threshold parameter. We also set trend preservation mode to be 'mixed'. Thus, we set the parameter object directly specifying:

* Lower bound = 0
* Lower threshold = 0.0000011574
* Trend preservation = mixed
* Distribution = gamma

Otherwise, we are keeping the default parameter values for the bias adjustment.

In [7]:
params = basd.Parameters(lower_bound=0, lower_threshold=0.0000011574,
                         trend_preservation='mixed', distribution='gamma',
                         if_all_invalid_use=0)
ba = basd.Adjustment(pr_obs_hist, pr_sim_hist, pr_sim_fut,
                     'pr', params, remap_grid=True )

#### Running Bias Adjustment
Here we run the bias adjustment for just one grid cell. We pass in a tuple with the indexes which we wish to adjust, (0,0), which in this case correspond to the coordinates 53.5 N, 12.5 E.

In [8]:
loc = dict(lat=1, lon=1)
sim_fut_ba_loc = ba.adjust_bias_one_location(loc)

In [9]:
pr_sim_fut_ba = ba.adjust_bias(n_jobs=-1)
#ba.save_adjustment_nc('../../../Documents/pr_sim_fut_ba.nc')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:   11.6s remaining:   11.6s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   11.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   11.7s finished


In [13]:
pr_sim_fut['pr'][loc].to_numpy()

array([6.4610385e-06, 2.3013397e-06, 1.0377765e-06, ..., 2.4182818e-06,
       4.4520571e-06, 8.5363882e-07], dtype=float32)

In [12]:
pr_sim_fut_ba['pr'][loc].to_numpy()

array([2.3071223e-06, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00], dtype=float32)

In [11]:
sim_fut_ba_loc.time_series

,time,lat,lon,spatial_ref,pr,Source
0,2065-01-01,52.5,13.5,0,2.307122e-06,Simulated Future Bias Adjusted
1,2065-01-02,52.5,13.5,0,0.000000e+00,Simulated Future Bias Adjusted
2,2065-01-03,52.5,13.5,0,0.000000e+00,Simulated Future Bias Adjusted
3,2065-01-04,52.5,13.5,0,2.959977e-05,Simulated Future Bias Adjusted
4,2065-01-05,52.5,13.5,0,2.926566e-05,Simulated Future Bias Adjusted
...,...,...,...,...,...,...
52589,2014-12-27,52.5,13.5,0,2.245260e-07,Simulated Historical
52590,2014-12-28,52.5,13.5,0,3.433956e-05,Simulated Historical
52591,2014-12-29,52.5,13.5,0,8.525437e-05,Simulated Historical
52592,2014-12-30,52.5,13.5,0,2.469289e-05,Simulated Historical
